In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql._

val spark = SparkSession.builder()
.appName("nulls")
.getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://d281b57a8bf3:4041
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1706658159423)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4d01c9ac


In [2]:
val mv = spark.read.option("inferSchema", "true").json("./dataset/movies.json")
mv.show(5)

+--------------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|       Creative_Type|Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+--------------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|                NULL|    NULL|   Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|      The Land Girls|        NULL|  146083|         146083|
|                NULL|    NULL|     

mv: org.apache.spark.sql.DataFrame = [Creative_Type: string, Director: string ... 14 more fields]


In [11]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

val mv1 = mv
.select(col("Title"), col("Rotten_Tomatoes_Rating"), to_date(col("Release_Date"), "dd-MMM-yy").as("Actual_Date"))
.filter(col("Rotten_Tomatoes_Rating").isNull)

mv1.show(5, false)

+--------------------------+----------------------+-----------+
|Title                     |Rotten_Tomatoes_Rating|Actual_Date|
+--------------------------+----------------------+-----------+
|The Land Girls            |NULL                  |1998-06-12 |
|First Love, Last Rites    |NULL                  |1998-08-07 |
|I Married a Strange Person|NULL                  |1998-08-28 |
|Mississippi Mermaid       |NULL                  |1999-01-15 |
|Following                 |NULL                  |1999-04-04 |
+--------------------------+----------------------+-----------+
only showing top 5 rows



mv1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Title: string, Rotten_Tomatoes_Rating: bigint ... 1 more field]


In [16]:
val mv2 = mv
.select(
    col("Major_Genre"),
    col("Rotten_Tomatoes_Rating"), 
    col("IMDB_Rating"), 
    coalesce(col("Major_Genre"), col("Rotten_Tomatoes_Rating"), col("IMDB_Rating")).as("firstNotNull")
)

mv2.show()

+-----------+----------------------+-----------+------------+
|Major_Genre|Rotten_Tomatoes_Rating|IMDB_Rating|firstNotNull|
+-----------+----------------------+-----------+------------+
|       NULL|                  NULL|        6.1|         6.1|
|      Drama|                  NULL|        6.9|       Drama|
|     Comedy|                  NULL|        6.8|      Comedy|
|     Comedy|                    13|       NULL|      Comedy|
|      Drama|                    62|        3.4|       Drama|
|       NULL|                  NULL|       NULL|        NULL|
|       NULL|                  NULL|        7.7|         7.7|
|     Comedy|                  NULL|        3.8|      Comedy|
|       NULL|                    25|        5.8|          25|
|       NULL|                    86|        7.0|          86|
|       NULL|                    81|        7.0|          81|
|    Musical|                    84|        7.5|     Musical|
|       NULL|                    97|        8.4|          97|
|       

mv2: org.apache.spark.sql.DataFrame = [Major_Genre: string, Rotten_Tomatoes_Rating: bigint ... 2 more fields]


In [17]:
mv2.orderBy(col("IMDB_Rating").desc_nulls_last).show()

+-----------------+----------------------+-----------+-----------------+
|      Major_Genre|Rotten_Tomatoes_Rating|IMDB_Rating|     firstNotNull|
+-----------------+----------------------+-----------+-----------------+
|             NULL|                   100|        9.2|              100|
|            Drama|                    88|        9.2|            Drama|
|Thriller/Suspense|                    87|        9.1|Thriller/Suspense|
|             NULL|                  NULL|        9.0|              9.0|
|            Drama|                  NULL|        8.9|            Drama|
|             NULL|                    96|        8.9|               96|
|            Drama|                    94|        8.9|            Drama|
|            Drama|                    97|        8.9|            Drama|
|           Action|                    93|        8.9|           Action|
|        Adventure|                    99|        8.9|        Adventure|
|            Drama|                    97|        8

In [20]:
// drop na

mv2.na.drop().show(5)

+-----------+----------------------+-----------+------------+
|Major_Genre|Rotten_Tomatoes_Rating|IMDB_Rating|firstNotNull|
+-----------+----------------------+-----------+------------+
|      Drama|                    62|        3.4|       Drama|
|    Musical|                    84|        7.5|     Musical|
|      Drama|                    57|        7.0|       Drama|
|     Comedy|                    33|        5.6|      Comedy|
|      Drama|                    77|        7.1|       Drama|
+-----------+----------------------+-----------+------------+
only showing top 5 rows



In [25]:
mv2.na.fill(0).show(5) // 타입이 맞는것만 fill 된다.

+-----------+----------------------+-----------+------------+
|Major_Genre|Rotten_Tomatoes_Rating|IMDB_Rating|firstNotNull|
+-----------+----------------------+-----------+------------+
|       NULL|                     0|        6.1|         6.1|
|      Drama|                     0|        6.9|       Drama|
|     Comedy|                     0|        6.8|      Comedy|
|     Comedy|                    13|        0.0|      Comedy|
|      Drama|                    62|        3.4|       Drama|
+-----------+----------------------+-----------+------------+
only showing top 5 rows



In [29]:
mv2.na.fill(0, Array("IMDB_Rating", "Rotten_Tomatoes_Rating")).show()

+-----------+----------------------+-----------+------------+
|Major_Genre|Rotten_Tomatoes_Rating|IMDB_Rating|firstNotNull|
+-----------+----------------------+-----------+------------+
|       NULL|                     0|        6.1|         6.1|
|      Drama|                     0|        6.9|       Drama|
|     Comedy|                     0|        6.8|      Comedy|
|     Comedy|                    13|        0.0|      Comedy|
|      Drama|                    62|        3.4|       Drama|
|       NULL|                     0|        0.0|        NULL|
|       NULL|                     0|        7.7|         7.7|
|     Comedy|                     0|        3.8|      Comedy|
|       NULL|                    25|        5.8|          25|
|       NULL|                    86|        7.0|          86|
|       NULL|                    81|        7.0|          81|
|    Musical|                    84|        7.5|     Musical|
|       NULL|                    97|        8.4|          97|
|       

In [34]:
// 각 컬럼별로 fill na 조건 설정
mv2.na.fill(Map(
    "Major_Genre" -> "UNKNOWN",
    "Rotten_Tomatoes_Rating" -> 11111,
    "IMDB_Rating" -> 22222,
    "firstNotNull" -> "ALLNULL"
)).show()

+-----------+----------------------+-----------+------------+
|Major_Genre|Rotten_Tomatoes_Rating|IMDB_Rating|firstNotNull|
+-----------+----------------------+-----------+------------+
|    UNKNOWN|                 11111|        6.1|         6.1|
|      Drama|                 11111|        6.9|       Drama|
|     Comedy|                 11111|        6.8|      Comedy|
|     Comedy|                    13|    22222.0|      Comedy|
|      Drama|                    62|        3.4|       Drama|
|    UNKNOWN|                 11111|    22222.0|     ALLNULL|
|    UNKNOWN|                 11111|        7.7|         7.7|
|     Comedy|                 11111|        3.8|      Comedy|
|    UNKNOWN|                    25|        5.8|          25|
|    UNKNOWN|                    86|        7.0|          86|
|    UNKNOWN|                    81|        7.0|          81|
|    Musical|                    84|        7.5|     Musical|
|    UNKNOWN|                    97|        8.4|          97|
|    UNK

In [10]:
// https://jaynamm.tistory.com/entry/Spark-SQL-function-ifnull-nullif-nvl-nvl2

mv.selectExpr(
    "Title",
    "IMDB_Rating",
    "Rotten_Tomatoes_Rating",
    "ifnull(Rotten_Tomatoes_Rating, IMDB_Rating * 10) as ifnull", // 첫번째가 null이면 두번쨰 value 반환
    "nvl(Rotten_Tomatoes_Rating, IMDB_Rating * 10) as nvl", // 첫번째가 null이면 두 번쨰 value 반환
    "nullif(Rotten_Tomatoes_Rating, IMDB_Rating * 10) as nullif", // 두 값이 같으면 null, 다르면 첫번째 값 반환
    "nvl2(Rotten_Tomatoes_Rating, IMDB_Rating * 10, 0.0) as nvl2", // 첫번째가 not null -> 두번째, null -> 세번째
    "nanvl(Rotten_Tomatoes_Rating, IMDB_Rating * 10) as nanvl"
).show()

+--------------------+-----------+----------------------+------+----+------+----+-----+
|               Title|IMDB_Rating|Rotten_Tomatoes_Rating|ifnull| nvl|nullif|nvl2|nanvl|
+--------------------+-----------+----------------------+------+----+------+----+-----+
|      The Land Girls|        6.1|                  NULL|  61.0|61.0|  NULL| 0.0| NULL|
|First Love, Last ...|        6.9|                  NULL|  69.0|69.0|  NULL| 0.0| NULL|
|I Married a Stran...|        6.8|                  NULL|  68.0|68.0|  NULL| 0.0| NULL|
|Let's Talk About Sex|       NULL|                    13|  13.0|13.0|    13|NULL| 13.0|
|                Slam|        3.4|                    62|  62.0|62.0|    62|34.0| 62.0|
| Mississippi Mermaid|       NULL|                  NULL|  NULL|NULL|  NULL| 0.0| NULL|
|           Following|        7.7|                  NULL|  77.0|77.0|  NULL| 0.0| NULL|
|             Foolish|        3.8|                  NULL|  38.0|38.0|  NULL| 0.0| NULL|
|             Pirates|        5.

## 다른 Method들 실습

In [4]:
mv.show(3)

+-------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+------+--------------------+------------+--------+---------------+
|Creative_Type|Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+-------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+------+--------------------+------------+--------+---------------+
|         NULL|    NULL|   Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|  NULL|      The Land Girls|        NULL|  146083|         146083|
|         NULL|    NULL|     Strand|        6.9|       207|          R|      Drama|           300000|    7-Aug-98|  

In [7]:
mv.select(col("Title"), encode(col("Title"), "UTF-8").as("Encoded")).show(5, false)

+--------------------------+-------------------------------------------------------------------------------+
|Title                     |Encoded                                                                        |
+--------------------------+-------------------------------------------------------------------------------+
|The Land Girls            |[54 68 65 20 4C 61 6E 64 20 47 69 72 6C 73]                                    |
|First Love, Last Rites    |[46 69 72 73 74 20 4C 6F 76 65 2C 20 4C 61 73 74 20 52 69 74 65 73]            |
|I Married a Strange Person|[49 20 4D 61 72 72 69 65 64 20 61 20 53 74 72 61 6E 67 65 20 50 65 72 73 6F 6E]|
|Let's Talk About Sex      |[4C 65 74 27 73 20 54 61 6C 6B 20 41 62 6F 75 74 20 53 65 78]                  |
|Slam                      |[53 6C 61 6D]                                                                  |
+--------------------------+-------------------------------------------------------------------------------+
only showing top 5 

In [15]:
mv.select(
    col("Title"),
    encode(col("Title"), "UTF-8").as("Encoded"),
    decode(col("Encoded"), "UTF-8").as("Decoded"),
    expr("spark_partition_id() as partition_id"),
    input_file_name()
).show(5, false)

+--------------------------+-------------------------------------------------------------------------------+--------------------------+------------+-------------------------------------------+
|Title                     |Encoded                                                                        |Decoded                   |partition_id|input_file_name()                          |
+--------------------------+-------------------------------------------------------------------------------+--------------------------+------------+-------------------------------------------+
|The Land Girls            |[54 68 65 20 4C 61 6E 64 20 47 69 72 6C 73]                                    |The Land Girls            |0           |file:///workspace/spark/dataset/movies.json|
|First Love, Last Rites    |[46 69 72 73 74 20 4C 6F 76 65 2C 20 4C 61 73 74 20 52 69 74 65 73]            |First Love, Last Rites    |0           |file:///workspace/spark/dataset/movies.json|
|I Married a Strange Person|[49 20 